# 一、感知机

数据需要线性（超平面）可分，如果不可分则模型在训练时会剧烈震荡。

## 1、建模公式

$$y^i = sign(W^TX^i+b)$$

<font color="red">假设有样本$I$个，每个样本对应$X^i$的特征向量，每个样本对应一个$y^i$类标。</font>

X是特征向量=$[x_1,x_2,x_3...x_t]$

W是权重矩阵=$[w_1,w_2,w_3...w_t]$

y是类标，一共有$I$个。

$sign(x)$或者$Sign(x)$叫做符号函数，在数学和计算机运算中，其功能是取某个数的符号（正或负）：

$$
sign(x)=
\begin{cases}
-1,\quad x\leq 0\\
1,\quad x>0
\end{cases}
\tag{1}
$$

## 2、损失函数

感知机函数是误分类点到分界线，或者分类的超平面的距离。
n维空间$R^n$中一点$X^i$到超平面$W^T*X^i+b=0$的距离公式：

$$\frac{|W^T*X^i+b|}{||W^T||}$$

感知器因为sign函数的原因有如下特点，对于误分类的点，有$y^i(W^T*X^i+b)<0$。而距离总是大于0的，因此，某个误分类的点到超平面的距离表示为：

$$\frac{-y^i(W^T*X^i+b)}{||W^T||}$$

||W||是一个常数，对最小化损失函数L(W,b)没有影响，故忽略之。

假设所有的误分类点都在集合M中，得到的损失函数如下：

$$L(w,b)=-\sum_{X^i}y^i(W^T*X^i+b)$$


## 3、梯度下降

有了损失函数了，就利用梯度下降法，不断调整w和b，从而不断减少损失即可。

对L(w,b)进行求导，来获取每次权重的更新量，这里的b可以变为$w_0x_0$，所以

$$L(w)=-\sum_{X^i}y^i(W^T*X^i)$$

求导得到：

$$\frac{\partial L(w)}{\partial w}=-\sum_{X^i}y^iX^i$$

在感知机中是一次只选择一个误分类点对W进行迭代更新，选择梯度的负方向进行更新，故更新的公式如下所示

$$W\leftarrow W-\eta(-y^iX^i)$$

$\eta$是学习率，取值范围是[0,1]

## 4、训练步骤

1. 读入训练样本$(X^i,y^i)$，执行预测$\hat{y}=sign(W.X^i)$
2. 如果$\hat{y}\neq y^i$，则更新参数$W\leftarrow W+ηy^iX^i$

说明：
在训练集的每个样本上执行1和2称为一次在线学习，把训练集完整的学习一遍称为一次迭代（epoch）

## 5、缺点

1. 由于感知机是一个在线学习模型，学习一个实例后就会更新整个模型，假设有10000个实例，前9999个实例都能正确预测，说明模型已经接近完美，但是最后一个实例是噪声点，那么就会预测错误从而更新模型，反而导致模型准确率下降。

## 6、解决方案

1. 创造更多特征，将样本映射到更高维的空间，使其线性可分。
2. 切换到其他训练算法，比如支持向量机等。
3. 对感知器算法做优化，使用投票感知机或平均感知机。

# 二、投票感知机和平均感知机

## 1、投票感知机

既然每次迭代都产生一个新模型，不知道哪个更好。那么让新模型与旧模型决斗:

在另一个样本集上评估一下每个模型的准确率，若新模型更准则替代旧模型，否则不更新。这么做存在漏洞，因为一般而言新模型总是比旧模型更准。再改进一下，每次迭代的模型都保留，准确率也保留。预测时每个模型都给出自己的结果，乘以它的准确率加权平均值作为最终结果，这样的补丁算法称为投票感知机。因为每正确预测一个样本， 该模型就得了一票，票数最多的模型对结果的影响最大。

## 2、平均感知机

投票感知机要求存储多个模型及加权，计算开销较大。更实际的做法是平均感知机，取多个模型的权重的平均。其优势在于预测时不再需要保存多个模型，而是在训练结束后将所有模型平均，只保留平均后的模型。

平均感知机在实现上也有技巧，那就是训练时也不保留多个模型，而是记录模型中会人参数在所有迭代时刻的总和$sum_d=\sum_t^TW_d^t$。将总和除以迭代次数，就能得到该参数的平均值$W_d=\frac{sum_d}{T}$，但如何累计权重也是个技术活。对于大部分训练实例，都只会引起特定的几个权重变化，其他特征函数的权重都不会变化。假设一共1万个参数，迭代100次就得执行100万次加法，效率负担很重。因此我们不应该盲目地将这些不变的权重也累加起来。

为此，进一步优化算法，记录每个参数w。上次更新的时间戳$t_1$。当我们在$t_2$时刻再次更新它的时候，它的累加值的增量就可以用一次乘法来解决了 : $sum_d \leftarrow sum_d+(t_2-t_1)*w_d^{t_1}$。
于是，对$w_i$来讲从$t_1$到$t_2$之间都不需要再做加法了，一下子节省了不少时间。

总结一下，平均感知机算法形式化描述如下。

1. 为每个参数$w_d$初始化累计量$sum_d=0$，上次更新时刻$time_d=0$，当前时刻t=0。

2. $t\leftarrow t+1$，读入训练样本$(X^i,y^i)$，执行预测$\hat{y}=sign(W.X^i)$。

3. 如果$\hat{y}\neq y^i$，则对所有需更新(即$X_d^i \neq 0$)的$W_d$执行：

    更新$sum_d\leftarrow sum_d + (t-time_d)*W_d$
    
    更新$time_d\leftarrow t$
    
    更新$W_d\leftarrow W_d+y^iX^i$
    
    
4. 训练指定迭代次数后计算平均值：$W_d=\frac{sum_d}{T}$

# 三、结构化预测问题

## 1、结构化定义

自然语言处理问题大致可分为两类，一类是分类问题，另一种就是结构化预测问题。

序列标注只是结构化预测的一个特例，对感知机稍作扩展，分类器就能支持结构化预测。

信息的层次结构特点称作结构化。那么结构化预测则是预测对象结构的一类监督学习问题。相应的模型训练过程称作结构化学习。

分类问题的预测结果是一个决策边界，回归问题的预测结果是一个实数变量，而结构化预测的结果则是一个完整的结构。可见结构化预测难度更高。


自然语言处理中有许多任务是结构化预测，比如

序列标注预测结构是一整个序列， 

句法分析预测结构是一棵句法树，

机器翻译预测结构是一段完整的译文。 

这些结构由许多部分构成，最小的部分虽然也是分类问题(比如中文分词时每个字符分类为{B.M,E,S} ),但必须考虑结构
整体的合理程度。所谓合理程度，通常用模型给出的分值或概率衡量。

## 2、结构化预测与学习的流程

结构化预测的过程就是给定一个模型$\lambda$及打分函数$score_{\lambda}(\boldsymbol{x},\boldsymbol{y})$，利用打分函数给一此备选结构打分，选择分数最高的结构作为预测输出，如下式所示:

$$\hat{\boldsymbol{y}}=arg \underset{\boldsymbol{y}\in Y}{max} score_{\lambda}(\boldsymbol{x},\boldsymbol{y})$$

<font color="red">在这里$\boldsymbol{y}$是一个矩阵，是所有的训练样本的类标，例如：分词，每一句话的每一个字都会对应一个类标。$\boldsymbol{x}$是一个矩阵，是所有的训练样本的特征，例如：分词，每一句话的每一个字都会有特征。</font>

其中，Y是备选结构的集合。备选结构可以是解空间的全集，也可以是一个子集。

既然结构化预测就是搜索得分最高的结构$\hat{y}$，那么结构化学习的目标就是想方设法让正确答案$\boldsymbol{y}^i$的得分最高。如此$\hat{y}=\boldsymbol{y}^i$，这次预测就是正确的。如何达到这个目标？不同的模型有不同的算法，对于线性模型，训练算法为结构化感知机。

# 四、线性模型的结构化感知机算法(Structured Perceptron, SP)

## 1、打分函数

要让线性模型支持结构化预测，必须先设计打分函数。打分函数的输入有两个缺一不可的参数：特征$\boldsymbol{x}$和结构$\boldsymbol{y}$。但之前介绍的线性模型的”打分函数“只接受一个自变量$\boldsymbol{x}$：

$$f(\boldsymbol{x})=\boldsymbol{w}.\boldsymbol{x}$$

怎么把$\boldsymbol{y}$也考虑进去呢？做法是定义新的特征函数$\phi(\boldsymbol{x},\boldsymbol{y})$，把结构$\boldsymbol{y}$也作为一种特征，输出新的"结构化特征向量"，$\phi(\boldsymbol{x},\boldsymbol{y})\in \mathbb{R}^{D\times1}$。新特征向量与权重向量做点积后，就得到一个标量，将其作为分数:

$$score(\boldsymbol{x},\boldsymbol{y})=\boldsymbol{w}.\phi(\boldsymbol{x},\boldsymbol{y})$$

打分函数有了，取分值最大的结构作为预测结果，得到结构化预测函数:

$$\hat{\boldsymbol{y}}=arg \underset{\boldsymbol{y}\in Y}{max}(\boldsymbol{w}.\phi(\boldsymbol{x},\boldsymbol{y}))$$

预测函数与线性分类器的决策函数很像，都是权重向量点积特征向量。那么感知机算法也可以拓展复用，得到线性模型的结构化学习算法。



## 2、损失函数

<font color="red">我利用权重更新的结果，倒推损失函数，当然不确定是否对。</font>

损失函数定义为正确类标的得分减掉错误类标的得分越来越大：

$$L(\boldsymbol{w})=\sum_{\boldsymbol{x}^i}\left(\boldsymbol{w}.\phi(\boldsymbol{x}^i,\boldsymbol{y}^i)-\boldsymbol{w}.\phi(\boldsymbol{x}^i,\hat{\boldsymbol{y}})\right)$$

求导：

$$\frac{\partial{L(\boldsymbol{w})}}{\partial{\boldsymbol{w}}}=\sum_{\boldsymbol{x}^i}\left(\phi(\boldsymbol{x}^i,\boldsymbol{y}^i)-\phi(\boldsymbol{x}^i,\hat{\boldsymbol{y}})\right)$$




## 3、训练步骤

1. 读入样本($\boldsymbol{x},\boldsymbol{y}$)，执行结构化预测$\hat{\boldsymbol{y}}=arg \underset{\boldsymbol{y}\in Y}{max}(\boldsymbol{w}.\phi(\boldsymbol{x},\boldsymbol{y}))$。

2. 与正确答案对比，若$\hat{\boldsymbol{y}}\neq \boldsymbol{y}^i$，则更新参数：

奖励正确答案触发的特征函数的权重

$$\boldsymbol{w}\leftarrow \boldsymbol{w}+\phi(\boldsymbol{x}^i,\boldsymbol{y}^i)$$

惩罚错误结果触发的特征函数的权重

$$\boldsymbol{w}\leftarrow \boldsymbol{w}-\phi(\boldsymbol{x}^i,\hat{\boldsymbol{y}})$$

奖惩可以合并到一个式子里：

$$\boldsymbol{w}\leftarrow \boldsymbol{w}+\phi(\boldsymbol{x}^i,\boldsymbol{y}^i)-\phi(\boldsymbol{x}^i,\hat{\boldsymbol{y}})$$

还可以调整学习率: 

$$\boldsymbol{w}\leftarrow \boldsymbol{w}+\eta\left(\phi(\boldsymbol{x}^i,\boldsymbol{y}^i)-\phi(\boldsymbol{x}^i,\hat{\boldsymbol{y}})\right)$$

相较于感知机算法，结构化感知机的不同点无非在于

1. 修改了特征向量
2. 参数更新赏罚分明

<font color="red">**注意：**

结构化感知机的权重更新，例如：分词，是对一句话中的每一个字进行预测类标，然后利用维特比（viterbi）算法计算最大概率的整句话的每个字类标，然后在对比每一个字的类标时候，如果预测和实际不一致，则更新权重。直到这句话的每个字的特征权重更新完成，再利用上一句更新后的权重，进行下一句话的类标预测，如此往复。【其实这就是每个句子训练或者更新一次权重，而crf是整个数据集作为一次迭代，做一次权重的训练或更新】</font>

```java
AveragedPerceptron model;
model = new AveragedPerceptron(immutableFeatureMap);
final double[] total = new double[model.parameter.length];
final int[] timestamp = new int[model.parameter.length];
int current = 0;
for (int iter = 1; iter <= maxIteration; iter++)
{
    long now = currentTimeMillis();
    Utility.shuffleArray(instances);
    for (Instance instance : instances)
    {
        ++current;
        int[] guessLabel = new int[instance.length()];
        model.viterbiDecode(instance, guessLabel);
        for (int i = 0; i < instance.length(); i++)
        {
            int[] featureVector = instance.getFeatureAt(i);
            int[] goldFeature = new int[featureVector.length];
            int[] predFeature = new int[featureVector.length];
            for (int j = 0; j < featureVector.length - 1; j++)
            {
                goldFeature[j] = featureVector[j] * tagSet.size() + instance.tagArray[i];
                predFeature[j] = featureVector[j] * tagSet.size() + guessLabel[i];
            }
            goldFeature[featureVector.length - 1] = (i == 0 ? tagSet.bosId() : instance.tagArray[i - 1]) * tagSet.size() + instance.tagArray[i];
            predFeature[featureVector.length - 1] = (i == 0 ? tagSet.bosId() : guessLabel[i - 1]) * tagSet.size() + guessLabel[i];
            model.update(goldFeature, predFeature, total, timestamp, current);
        }
    }

    // 在开发集上校验
    accuracy = trainingFile.equals(developFile) ? IOUtility.evaluate(instances, model) : evaluate(developFile, model);
    out.printf("Iter#%d - ", iter);
    printAccuracy(accuracy);
    out.println("耗时:"+ String.valueOf(currentTimeMillis() - now));
}
// 平均
model.average(total, timestamp, current);
accuracy = trainingFile.equals(developFile) ? IOUtility.evaluate(instances, model) : evaluate(developFile, model);
out.print("AP - ");
printAccuracy(accuracy);
logger.start("以压缩比 %.2f 保存模型到 %s ... ", compressRatio, modelFile);
model.save(modelFile, immutableFeatureMap.featureIdMap.entrySet(), compressRatio);
logger.finish(" 保存完毕\n");
if (compressRatio == 0) return new Result(model, accuracy);

```

在HanLP中，结构化感知机的成员变量也与感知机一致，只是要求上层应用负责提取结构化特征，并且调用重载的参数更新方法。关键代码位于com.hankcs. hanlp.model.perceptron.
model.StructuredPerceptron#update :

```java

 /**
 * 根据答案和预测更新参数
 *
 * @param goldIndex    预测正确的特征函数（非压缩形式）
 * @param predictIndex 命中的特征函数
 */
public void update(int[] goldIndex, int[] predictIndex, double[] total, int[] timestamp, int current)
{
    for (int i = 0; i < goldIndex.length; ++i)
    {
        if (goldIndex[i] == predictIndex[i])
            continue;
        else
        {
            update(goldIndex[i], 1, total, timestamp, current);  //正确的那个权重加分
            if (predictIndex[i] >= 0 && predictIndex[i] < parameter.length)
                update(predictIndex[i], -1, total, timestamp, current); //错误的那个权重减分
            else
            {
                throw new IllegalArgumentException("更新参数时传入了非法的下标");
            }
        }
    }
}

```